In [1]:
import pandas as pd
from os.path import join
from sklearn.utils import shuffle

emotions = ["happy", "sad", ["disgust", "disgust2"], "angry", "fear", "surprise"]
dir_path = "gathering/ja_tweets_sentiment"
size = 60000
df = []
for i, es in enumerate(emotions):
    if isinstance(es, list):
        for e in es:
            data = shuffle(pd.read_json(join(dir_path, "{}.json".format(e)))).iloc[:int(size/len(es))]
            data['label'] = i
            df.append(data)
    else:
        data = shuffle(pd.read_json(join(dir_path, "{}.json".format(es)))).iloc[:int(size)]
        data['label'] = i
        df.append(data)

df = pd.concat(df)
df.shape

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


(360000, 11)

In [2]:
df = shuffle(df)
X = df['text']
y = df['label']

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def preprocess(data, tokenizer, maxlen=280):
    return(pad_sequences(tokenizer.texts_to_sequences(data), maxlen=maxlen))

max_features=10000
maxlen = 280

y = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, y)

tokenizer = Tokenizer(num_words=max_features, filters="", char_level=True)
tokenizer.fit_on_texts(list(X_train))

X_train = preprocess(X_train, tokenizer)
X_test = preprocess(X_test, tokenizer)

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)

In [5]:
from keras.layers import Input, Dense, Embedding, Flatten
from keras.layers import SpatialDropout1D
from keras.layers import LSTM, Conv1D, MaxPooling1D, Bidirectional
from keras.models import Sequential

In [24]:
model = Sequential()
model.add(Embedding(max_features, 256, input_length=maxlen))
model.add(SpatialDropout1D(0.3))
model.add(Conv1D(32, 3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, 3, activation="relu"))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(6, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
epochs = 5
batch_size = 1000

In [26]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)

Train on 202500 samples, validate on 67500 samples
Epoch 1/5
202500/202500 [==============================] - 33s 162us/step - loss: 1.6053 - acc: 0.3136 - val_loss: 1.4775 - val_acc: 0.3943
Epoch 2/5
202500/202500 [==============================] - 31s 154us/step - loss: 1.4194 - acc: 0.4351 - val_loss: 1.3953 - val_acc: 0.4524
Epoch 3/5
202500/202500 [==============================] - 31s 155us/step - loss: 1.3494 - acc: 0.4741 - val_loss: 1.3574 - val_acc: 0.4699
Epoch 4/5
202500/202500 [==============================] - 31s 155us/step - loss: 1.3068 - acc: 0.4950 - val_loss: 1.3448 - val_acc: 0.4780
Epoch 5/5
202500/202500 [==============================] - 31s 155us/step - loss: 1.2756 - acc: 0.5105 - val_loss: 1.3335 - val_acc: 0.4815


In [27]:
import pickle

with open("tokenizer_cnn_ja.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [28]:
model.save("model_2018-08-29-10:37.h5")

In [29]:
import numpy as np
from sklearn.metrics import classification_report

emolabels = []
for e in emotions:
    if isinstance(e, list):
        emolabels.append(e[0])
    else:
        emolabels.append(e)
        
y_preds = model.predict(X_test)
y_preds = np.argmax(y_preds, axis=1)
y_true = np.argmax(y_test, axis=1)

In [30]:
print(classification_report(y_true, y_preds, target_names=emolabels))

             precision    recall  f1-score   support

      happy       0.51      0.60      0.55     15200
        sad       0.58      0.53      0.55     14763
    disgust       0.43      0.38      0.41     14897
      angry       0.42      0.55      0.47     14948
       fear       0.52      0.47      0.49     15211
   surprise       0.47      0.38      0.42     14981

avg / total       0.49      0.49      0.48     90000



In [33]:
epochs = 2
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)

Train on 202500 samples, validate on 67500 samples
Epoch 1/2
202500/202500 [==============================] - 31s 154us/step - loss: 1.2488 - acc: 0.5217 - val_loss: 1.3344 - val_acc: 0.4821
Epoch 2/2
202500/202500 [==============================] - 31s 154us/step - loss: 1.2261 - acc: 0.5329 - val_loss: 1.3269 - val_acc: 0.4856


In [34]:
y_preds = model.predict(X_test)
y_preds = np.argmax(y_preds, axis=1)
print(classification_report(y_true, y_preds, target_names=emolabels))

             precision    recall  f1-score   support

      happy       0.52      0.59      0.55     15200
        sad       0.57      0.53      0.55     14763
    disgust       0.41      0.43      0.42     14897
      angry       0.42      0.57      0.48     14948
       fear       0.55      0.44      0.49     15211
   surprise       0.49      0.37      0.42     14981

avg / total       0.49      0.49      0.49     90000



In [35]:
model.save("model_2018-08-29-10:43.h5")

In [38]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)

Train on 202500 samples, validate on 67500 samples
Epoch 1/2
202500/202500 [==============================] - 31s 153us/step - loss: 1.2061 - acc: 0.5414 - val_loss: 1.3375 - val_acc: 0.4846
Epoch 2/2
202500/202500 [==============================] - 31s 155us/step - loss: 1.1877 - acc: 0.5497 - val_loss: 1.3329 - val_acc: 0.4861


In [39]:
y_preds = model.predict(X_test)
y_preds = np.argmax(y_preds, axis=1)
print(classification_report(y_true, y_preds, target_names=emolabels))

             precision    recall  f1-score   support

      happy       0.55      0.56      0.55     15200
        sad       0.61      0.51      0.55     14763
    disgust       0.40      0.45      0.42     14897
      angry       0.41      0.58      0.48     14948
       fear       0.59      0.41      0.48     15211
   surprise       0.46      0.44      0.45     14981

avg / total       0.50      0.49      0.49     90000



In [40]:
model.save("model_2018-08-29-10:47.h5")

In [43]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)

Train on 202500 samples, validate on 67500 samples
Epoch 1/2
202500/202500 [==============================] - 31s 154us/step - loss: 1.1714 - acc: 0.5564 - val_loss: 1.3432 - val_acc: 0.4863
Epoch 2/2
202500/202500 [==============================] - 31s 154us/step - loss: 1.1574 - acc: 0.5630 - val_loss: 1.3455 - val_acc: 0.4840


In [45]:
y_preds = model.predict(X_test)
y_preds = np.argmax(y_preds, axis=1)
print(classification_report(y_true, y_preds, target_names=emolabels))

             precision    recall  f1-score   support

      happy       0.52      0.58      0.55     15200
        sad       0.52      0.57      0.55     14763
    disgust       0.42      0.40      0.41     14897
      angry       0.45      0.49      0.47     14948
       fear       0.51      0.47      0.49     15211
   surprise       0.48      0.41      0.44     14981

avg / total       0.49      0.49      0.49     90000



In [47]:
epochs=5
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)

Train on 202500 samples, validate on 67500 samples
Epoch 1/5
202500/202500 [==============================] - 31s 154us/step - loss: 1.1425 - acc: 0.5682 - val_loss: 1.3663 - val_acc: 0.4792
Epoch 2/5
202500/202500 [==============================] - 31s 154us/step - loss: 1.1310 - acc: 0.5732 - val_loss: 1.3526 - val_acc: 0.4825
Epoch 3/5
202500/202500 [==============================] - 31s 154us/step - loss: 1.1189 - acc: 0.5789 - val_loss: 1.3607 - val_acc: 0.4829
Epoch 4/5
202500/202500 [==============================] - 31s 155us/step - loss: 1.1065 - acc: 0.5837 - val_loss: 1.3625 - val_acc: 0.4852
Epoch 5/5
202500/202500 [==============================] - 31s 155us/step - loss: 1.0985 - acc: 0.5880 - val_loss: 1.3749 - val_acc: 0.4801


In [49]:
y_preds = model.predict(X_test)
y_preds = np.argmax(y_preds, axis=1)
print(classification_report(y_true, y_preds, target_names=emolabels))

             precision    recall  f1-score   support

      happy       0.57      0.50      0.53     15200
        sad       0.59      0.52      0.55     14763
    disgust       0.38      0.48      0.42     14897
      angry       0.43      0.52      0.47     14948
       fear       0.53      0.45      0.48     15211
   surprise       0.46      0.43      0.45     14981

avg / total       0.49      0.48      0.49     90000



In [50]:
examples = [
    "まじきもい、あいつ",
    "今日は楽しい一日だったよ",
    "ペットが死んだ、実に悲しい",
    "ふざけるな、死ね",
    "ストーカー怖い",
    "すごい！ほんとに！？",
    "葉は植物の構成要素です。"
]

targets = preprocess(examples, tokenizer, maxlen=maxlen)
print('\t'.join(emolabels))
for i, ds in enumerate(model.predict(targets)):
    print('\t'.join([str(round(100.0*d)) for d in ds]))

happy	sad	disgust	angry	fear	surprise
2.0	1.0	4.0	1.0	1.0	1.0
6.0	1.0	2.0	1.0	0.0	1.0
0.0	10.0	5.0	0.0	2.0	0.0
0.0	1.0	4.0	30.0	1.0	1.0
1.0	1.0	3.0	3.0	2.0	1.0
1.0	1.0	2.0	1.0	3.0	6.0
5.0	0.0	0.0	2.0	1.0	15.0


In [51]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_preds)

0.4827222222222222